## 第6章 数据加载、存储与文件格式

In [4]:
import pandas as pd
import numpy as np
np.random.seed(2019)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 读写文本格式的数据

In [2]:
df = pd.read_csv('./examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
pd.read_table('./examples/ex1.csv', sep = '.')    # sep，指定分隔符

/home/ubuntu/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,"a,b,c,d,message"
0,"1,2,3,4,hello"
1,"5,6,7,8,world"
2,"9,10,11,12,foo"


In [9]:
!cat ./examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [21]:
pd.read_csv('./examples/ex2.csv', header=None) # pandas分配默认的列名

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [20]:
pd.read_csv('./examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])  # 自定义列名

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [22]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('./examples/ex2.csv', names=names, index_col='message')  # index_col指定索引

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [23]:
parsed = pd.read_csv('./examples/csv_mindex.csv', index_col=['key1', 'key2']) # 层次化索引
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [14]:
!cat ./examples/ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [24]:
result = pd.read_table('./examples/ex3.txt', sep='\s+') # 传入正则表达式表示分隔符
result

/home/ubuntu/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [25]:
# 使用skiprows跳过文件的第一行、第三行和第四行
!cat ./examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [27]:
pd.read_csv('./examples/ex4.csv', skiprows=[0,2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [28]:
# 缺失值处理
!cat ./examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [29]:
result = pd.read_csv('./examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [30]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [32]:
# na_values 可以用一个列表或集合的字符串表示缺失值
result = pd.read_csv('./examples/ex5.csv', na_values='NULL')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [33]:
# 字典的各列可以使用不同的NA标记值
sentinels = {'message':['foo', 'NA'], 'something':['two']}
pd.read_csv('./examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### 逐块读取文本文件

In [35]:
# 设置pandas显示的更紧一些
pd.options.display.max_rows = 10

In [36]:
result = pd.read_csv('./examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [37]:
# 如果只想读取几行，通过nrows进行指定：
pd.read_csv('./examples/ex6.csv', nrows=8)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K


In [42]:
# 要逐块读取文件，可以指定chunksize(行数)
chunker = pd.read_csv('./examples/ex6.csv', chunksize=1000)
chunker

In [44]:
# read_csv返回的这个TextFileReader对象使你可以根据chunksize对文件进行逐块迭代
# 比如说我们可迭代处理ex6.csv，将值技术聚合到'key'列中：
chunker = pd.read_csv('./examples/ex6.csv', chunksize=1000)
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)
tot

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

### 将数据写出到文本格式

In [45]:
data = pd.read_csv('./examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [46]:
# 利用to_csv方法，将数据写到一个以逗号分割的文件
data.to_csv('./examples/out.csv')
!cat ./examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [51]:
# 还可以使用其他分隔符,sep指定
import sys
data.to_csv(sys.stdout, sep='*')

*something*a*b*c*d*message
0*one*1*2*3.0*4*
1*two*5*6**8*world
2*three*9*10*11.0*12*foo


In [52]:
# 缺失值在输出结果里会被表示为空字符串，可以na_rep表示为自定义的标记值
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [53]:
# 如果没有设置其他选项，则会写出行和列的标签，当然，然们也都可以被禁用
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [54]:
# 只写出一部分列，并以指定的顺序排列
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [57]:
# Series也有一个to_csv方法：
dates = pd.date_range('1/1/1990', periods=10)
ts = pd.Series(np.arange(10), index=dates)
ts.to_csv('./examples/tseries.csv')
!cat ./examples/tseries.csv

1990-01-01,0
1990-01-02,1
1990-01-03,2
1990-01-04,3
1990-01-05,4
1990-01-06,5
1990-01-07,6
1990-01-08,7
1990-01-09,8
1990-01-10,9


/home/ubuntu/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.


### 处理分隔符格式

有一些数据需要进行特殊处理

In [59]:
!cat ./examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [61]:
# 对于任何单字符分隔符文件，可以直接使用Python内置的csv模块。
# 将任意已打开的文件或文件型的对象传给csv.reader

import csv

f = open('./examples/ex7.csv')

reader = csv.reader(f)

In [62]:
# 对reader进行迭代将会为每行产生一个元祖（并溢出了所有的引号）
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


为了使数据格式合乎要求，需要对其整理

In [63]:
# 读取文件到一个多行列表
with open('./examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [64]:
# 将这些行分为标题行和数据行
header, values = lines[0], lines[1:]

In [65]:
# 使用字典构造式和zip(*values)，后者将行转置为列，创建数据列的字典
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

## JSON数据
JSON（JavaScript Object Notation的简称）已经成为通过HTTP
请求在Web浏览器和其他应⽤程序之间发送数据的标准格式之
⼀。它是⼀种⽐表格型⽂本格式（如CSV）灵活得多的数据格
式。

In [66]:

obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [67]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [69]:
# json.dumps则将Python对象转换成JSON格式：
asjson = json.dumps(result)

In [70]:
# 将（⼀个或⼀组）JSON对象转换为DataFrame或其他便于分析的数据结构
# 最简单⽅便的⽅式是：向DataFrame构造器传⼊⼀个字典的列表（就是原先的JSON对象），并选取数据字段的⼦集
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


In [71]:
!cat ./examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [74]:
# pandas.read_json可以⾃动将特别格式的JSON数据集转换为Series或DataFrame
# pandas.read_json的默认选项假设JSON数组中的每个对象是表格中的⼀⾏

data = pd.read_json('./examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [76]:
# 将数据从pandas输出到JSON，可以使⽤to_json
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


## XML和HTML： Web信息收集
Python有许多可以读写常⻅的HTML和XML格式数据的库，包括
lxml、 Beautiful Soup和html5lib。 lxml的速度⽐较快，但其它的
库处理有误的HTML或XML⽂件更好

pandas有⼀个内置的功能， read_html，它可以使⽤lxml和
Beautiful Soup⾃动将HTML⽂件中的表格解析为DataFrame对
象




In [78]:
tables = pd.read_html('./examples/fdic_failed_bank_list.html')
len(tables)

ImportError: lxml not found, please install it